# HANDS-ON PRACTICE SESSION 3: PRACTICAL CONSIDERATIONS

SO Basics of Neural Networks 2023 school at the IAA-CSIC. Nov 2023.  
Eduardo Sánchez Karhunen  (fesanchez@us.es)  
University of Seville. Spain.  Dept. of CS and AI

### Introduction

In the last session, a CNN was designed to tackle the Galaxy10 classification problem. The obtained accuracy was approx. 71%.

In this third practice we are going to apply some improvements to our CNNs. Basically:
* Early Stopping
* Drop-out
* Data-Augmentation

The final goal of these techniques is to obtain an improvement in the network classification results, fighting against the overfitting.

### 1. Loading libraries

In [ ]:
import numpy as np
import h5py
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# only execute this lines if the following error appears
# OMP: Error #15: Initializing libiomp5md.dll, but found libiomp5md.dll already initialized.

# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

### Exercise 0: Dataset preparation

As in the previous session, download the dataset to your local disk.

In [ ]:
# LINUX
!wget http://www.astro.utoronto.ca/~bovy/Galaxy10/Galaxy10.h5

# WINDOWS
# !curl "http://www.astro.utoronto.ca/~bovy/Galaxy10/Galaxy10.h5" --output Galaxy10.h5

--2023-11-12 22:47:55--  http://www.astro.utoronto.ca/~bovy/Galaxy10/Galaxy10.h5
Resolving www.astro.utoronto.ca (www.astro.utoronto.ca)... 128.100.89.92
Connecting to www.astro.utoronto.ca (www.astro.utoronto.ca)|128.100.89.92|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 210234548 (200M)
Saving to: ‘Galaxy10.h5.1’

Galaxy10.h5.1       100%[===================>] 200.50M  13.6MB/s    in 17s     

2023-11-12 22:48:12 (12.1 MB/s) - ‘Galaxy10.h5.1’ saved [210234548/210234548]



Convert the downloaded h5 files into two numpy arrays containing images and labels.

In [ ]:
hf = h5py.File('Galaxy10.h5', 'r')
labels, images = hf.get('ans')[()], hf.get('images')[()]

Check the shape of the dataset.

In [ ]:
n_images, *shape, n_channels = images.shape
print(f"number of images: {n_images}")
print(f"shape of each image: {shape}")
print(f"number of channels: {n_channels}")

number of images: 21785
shape of each image: [69, 69]
number of channels: 3


### Exercise 1. Create train and test datasets

Divide the dataset into two new datasets for:
* training: 90%
* test: 10%  

Use the `train_test_split` command from scikit_learn. Do not worry for the validation dataset. We will create it in a posterior step.

More info in: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

**Task:** Create the following arrays: `train_images`, `train_labels`, `test_images` and `test_labels`.

In [ ]:
# your code here




### Exercise 2. Initialize ImageGenerators

ImageGenerators are needed to apply data-augmentation. It can be taken advantage of them to rescale and split data into train and validation in the same step.

Remember:
* Scaling the pixel values is always needed. Use the param `rescale` to indicate de scaling value.
* As a network designer you must decide which augmentation techniques fit better to your concrete problem. Probably some of them are not useful and can produce confusion between classes. In this case you can safely use horizontal flip.

* We can also split validation data at this moment. Using `validation_split` argument to indicate the size. A typical split value is 0.2.

More info in: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

**Task:** Create an ImageDataGenerator to rescale, split validation data and apply horizontal flip aaugmentations.

In [ ]:
# your code here




### Exercise 3. Connect ImageGenerators with data arrays

Once created the generator, it must be applied to our data arrays. Depending of the data location, different methods can be applied:

* **flow_from_directory**: if data is stored in the typical folder structure (one folder for each class).
* **flow**: when data in stored in numpy arrays in memory. Our Galaxy10 dataset is enough small to load it into a numpy array in memory.

More info in: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow and https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_directory

**Task:** Create two generators `train_generator`and `validation_generator` connected with arrays `train_images` and `train_labels`.

Hint: in both cases, the input arguments are the same: train_images and train_labels. The third argument indicates if the desired gen is for training or for validation.


In [ ]:
# your code here




### Exercise 4. Building the CNN model

Remember to follow this same schema:

<br>

![picture](https://drive.google.com/uc?id=1k8aHjwHlRP3wgU-IAtpKbcvwqyvvyUwY)


The parameters of the different layers are:

A) Convolutional layer
* 12 kernels (3x3)
* Stride = 1. The kernel will be shifted pixel by pixel along the image.
* No-padding. In Tensorflow, this is indicated as padding = `valid`.

B) Pooling layer
* pool_size = 2x2
* stride = 2

C) Flatten layer  
D) Two dense layers with 500 and 100 neurons, respectively.

**Task:** create a `conv_1`, `pool_1`, `flatten_layer`, `dense_1`, `dense_2` and `output_layer` with the previous parameters.

In [ ]:
# your code here




### Exercise 5. Dropout layer

Dropout layers can be inserted almost in any position in the network. We will add three of them after the flatten and both dense layers.

More info: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout

**Task:** Create three dropout layers `do_1`, `do_2` and `do_3` with drop_rate 0.35.

In [ ]:
# your code here




### Exercise 6. Join all layers in a CNN

The first step is to create a `sequential` structure to be filled with the previously created layers.

More info in: https://www.tensorflow.org/api_docs/python/tf/keras/Sequential

**Task:** Collect all the previous layers in a `model`.

In [ ]:
# your code here




### Exercise 7. Model inspection

In the model summary we can check the three new dropout layers have been added. Of course, these layers have no parameters.

**Task:** show a summary of the network

In [ ]:
# your code here




### Exercise 8. Assign loss function and optimizer

**Task:** assign the classical parameters to the network

In [ ]:
# your code here




### Exercise 9. Create an early stopping callback

Remeber EarlyStopping has two parameters:
* patience
* restore_best_weights

**Task:** Create an early stopping callback `es_callback` with patience = 4. Restore best weights when training finishes.

In [ ]:
# your code here




### Exercise 10. Network training

Remember:
* Use early stopping in training via the `callbacks` parameter. It expects a list, even if there is only one callback
* Validation_split is no more needed because this task has been performed by our generators.
* Use as training and validation data the two previously created generators
* Increase the number of epochs e.g. 30 epochs.

More info: https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit

**Task:** Train the model with the selected number of epochs, callback and using the generators (save the training history)

In [ ]:
# your code here




### Exercise 11: Plot training history

It is a good practice to plot our training curves. Typically we compare the evolution of loss function (accuracy) vs epochs.

**Task:** Plot train and valid loss (and accuracy) evolution during training

In [ ]:
# your code here




### Exercise 12. Evaluate model accuracy

To evaluate the model accuracy in the test dataset, first you must rescale `test_images`. You can do it in the traditional way, as in the previous practices or creating a generator simply for scaling.

**Task:** Once trained the model, obtain the accuracy of the model on the test dataset.

In [ ]:
# your code here




### Exercise 13: Making predictions with the model

Given an input image, the `predict` method returns the output of the last layer. This output is interpreted as the probability of the image to correspond to each of the classes.

**Task:** Inject to the model 3 images from the test dataset to obtain their probabilities.

In [ ]:
# your code here




**Task:** Using `np.argmax` and the probabilities, decide which is the class of each image

In [ ]:
# your code here




**Task:** Finally, compare the predicted classes with the real ones. There should be approx. 75% of correct predictions.

In [ ]:
# your code here



